In [1]:
# POC for ny taxi

In [106]:
!pip install pandasql

     |████████████████████████████████| 1.5 MB 5.0 MB/s eta 0:00:01
     |████████████████████████████████| 92 kB 4.2 MB/s  eta 0:00:01
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26781 sha256=286e59257cad4754f0bbc2490c0c037f1d27a732a0a26007c5c8f5e266c30f76
  Stored in directory: /Users/Nikolaus.Werner/Library/Caches/pip/wheels/ed/8f/46/a383923333728744f01ba24adbd8e364f2cb9470a8b8e5b9ff
Successfully built pandasql
You should consider upgrading via the '/Users/Nikolaus.Werner/.local/share/virtualenvs/ny_taxi-NY3wYOp9/bin/python -m pip install --upgrade pip' command.


In [107]:
import pandas as pd
import glob
from pandasql import sqldf

In [108]:
# helpers
pysqldf = lambda q: sqldf(q, globals())

In [2]:
data_file_location = '../resources/data/*.parquet'

## Loading enriched preprocessed data

In [19]:
# trip data
all_trip_data_df = None
for source_file_name in glob.glob(data_file_location):
    if all_trip_data_df is None:
        all_trip_data_df = pd.read_parquet(source_file_name)
    else:
        pd.concat([all_trip_data_df, pd.read_parquet(source_file_name)], ignore_index=True)

In [110]:
all_trip_data_df

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,pickup_borough,pickup_zone,drop_off_borough,drop_off_zone,year,month
0,2019-01-01 00:46:40,2019-01-01 00:53:20,1,Manhattan,Manhattan Valley,Manhattan,Upper West Side South,2019,2019-01
1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,Manhattan,Upper West Side South,Manhattan,West Chelsea/Hudson Yards,2019,2019-01
2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,Manhattan,Upper East Side North,Manhattan,Upper East Side North,2018,2018-12
3,2018-11-28 15:52:25,2018-11-28 15:55:45,5,Queens,Queensbridge/Ravenswood,Queens,Queensbridge/Ravenswood,2018,2018-11
4,2018-11-28 15:56:57,2018-11-28 15:58:33,5,Queens,Queensbridge/Ravenswood,Queens,Queensbridge/Ravenswood,2018,2018-11
...,...,...,...,...,...,...,...,...,...
7667787,2019-01-31 23:57:36,2019-02-01 00:18:39,1,Manhattan,Yorkville West,Manhattan,Alphabet City,2019,2019-02
7667788,2019-01-31 23:32:03,2019-01-31 23:33:11,1,Queens,Queensbridge/Ravenswood,Queens,Queensbridge/Ravenswood,2019,2019-01
7667789,2019-01-31 23:36:36,2019-01-31 23:36:40,1,Unknown,NV,Unknown,NV,2019,2019-01
7667790,2019-01-31 23:14:53,2019-01-31 23:15:20,1,Unknown,NV,Queens,Astoria,2019,2019-01


In [112]:
all_trip_data_df = all_trip_data_df.dropna(subset=['pickup_zone', 'drop_off_zone']) # TODO: can be done in preproc as well
all_trip_data_df

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,pickup_borough,pickup_zone,drop_off_borough,drop_off_zone,year,month
0,2019-01-01 00:46:40,2019-01-01 00:53:20,1,Manhattan,Manhattan Valley,Manhattan,Upper West Side South,2019,2019-01
1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,Manhattan,Upper West Side South,Manhattan,West Chelsea/Hudson Yards,2019,2019-01
2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,Manhattan,Upper East Side North,Manhattan,Upper East Side North,2018,2018-12
3,2018-11-28 15:52:25,2018-11-28 15:55:45,5,Queens,Queensbridge/Ravenswood,Queens,Queensbridge/Ravenswood,2018,2018-11
4,2018-11-28 15:56:57,2018-11-28 15:58:33,5,Queens,Queensbridge/Ravenswood,Queens,Queensbridge/Ravenswood,2018,2018-11
...,...,...,...,...,...,...,...,...,...
7667787,2019-01-31 23:57:36,2019-02-01 00:18:39,1,Manhattan,Yorkville West,Manhattan,Alphabet City,2019,2019-02
7667788,2019-01-31 23:32:03,2019-01-31 23:33:11,1,Queens,Queensbridge/Ravenswood,Queens,Queensbridge/Ravenswood,2019,2019-01
7667789,2019-01-31 23:36:36,2019-01-31 23:36:40,1,Unknown,NV,Unknown,NV,2019,2019-01
7667790,2019-01-31 23:14:53,2019-01-31 23:15:20,1,Unknown,NV,Queens,Astoria,2019,2019-01


## Task 2.i

In [115]:
grouped = pysqldf("SELECT sum(passenger_count) as count, pickup_zone, drop_off_zone, month FROM all_trip_data_df GROUP BY pickup_zone, drop_off_zone, month")
grouped


,count,pickup_zone,drop_off_zone,month
0,22,Allerton/Pelham Gardens,Allerton/Pelham Gardens,2019-01
1,1,Allerton/Pelham Gardens,Bayside,2019-01
2,1,Allerton/Pelham Gardens,Bedford,2019-01
3,2,Allerton/Pelham Gardens,Bedford Park,2019-01
4,1,Allerton/Pelham Gardens,Borough Park,2019-01
...,...,...,...,...
33962,15387,Yorkville West,Yorkville East,2019-01
33963,2,Yorkville West,Yorkville East,2019-02
33964,1,Yorkville West,Yorkville West,2009-01
33965,17484,Yorkville West,Yorkville West,2019-01


In [119]:
# rank by k=50 per pickup_zone and month
grouped["rank"]=grouped.groupby(["month","pickup_zone"])["count"].rank(method="max", ascending=False)
ranked = grouped[grouped["rank"] >= 50.0]

In [120]:
# 2.i) Sample Queries
grouped[grouped["pickup_zone"] == "Allerton/Pelham Gardens"]

,count,pickup_zone,drop_off_zone,month,rank
0,22,Allerton/Pelham Gardens,Allerton/Pelham Gardens,2019-01,1.0
1,1,Allerton/Pelham Gardens,Bayside,2019-01,74.0
2,1,Allerton/Pelham Gardens,Bedford,2019-01,74.0
3,2,Allerton/Pelham Gardens,Bedford Park,2019-01,41.0
4,1,Allerton/Pelham Gardens,Borough Park,2019-01,74.0
...,...,...,...,...,...
69,7,Allerton/Pelham Gardens,Washington Heights South,2019-01,11.0
70,2,Allerton/Pelham Gardens,West Concourse,2019-01,41.0
71,1,Allerton/Pelham Gardens,Westchester Village/Unionport,2019-01,74.0
72,1,Allerton/Pelham Gardens,Williamsbridge/Olinville,2019-01,74.0


In [122]:
# 2.i) Sample Queries
# Which destination was reached by the most passengers from Chinatown in September 2019.
grouped[(grouped["rank"]==1.0) & (grouped["pickup_zone"] == "Chinatown")]

,count,pickup_zone,drop_off_zone,month,rank
4618,1019,Chinatown,Lower East Side,2019-01,1.0
4695,3,Chinatown,Upper East Side North,2019-02,1.0
